In [3]:
!pip install awswrangler

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
%%time
import awswrangler as wr
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
import sagemaker
from sagemaker import get_execution_role
import pandas as pd

role = get_execution_role()
region = boto3.Session().region_name

containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

bucket = 'ads508team7'
prefix = 'jump_start'

CPU times: user 281 ms, sys: 56.5 ms, total: 338 ms
Wall time: 390 ms


In [5]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[region],role, instance_count=1, instance_type='ml.m5.xlarge',output_path='s3://{}/{}/output'.format(bucket, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=40)

In [6]:
%%time
xgb.fit({'train': s3_input_train})

2022-04-19 05:48:28 Starting - Starting the training job...
2022-04-19 05:48:58 Starting - Preparing the instances for trainingProfilerReport-1650347308: InProgress
.........
2022-04-19 05:50:18 Downloading - Downloading input data.........
2022-04-19 05:51:59 Training - Training image download completed. Training in progress.Arguments: train
Arguments: train
Arguments: train
[2022-04-19:05:51:51:INFO] Running distributed xgboost training.
[2022-04-19:05:51:51:INFO] Number of hosts: 5, master IP address: 10.2.105.73, host IP address: 10.2.118.156.
[2022-04-19:05:51:51:INFO] Finished Yarn configuration files setup.
Arguments: train
[2022-04-19:05:51:51:INFO] Running distributed xgboost training.
[2022-04-19:05:51:51:INFO] Number of hosts: 5, master IP address: 10.2.105.73, host IP address: 10.2.76.44.
[2022-04-19:05:51:51:INFO] Finished Yarn configuration files setup.
starting datanode, logging to /opt/amazon/hadoop/logs/hadoop--datanode-ip-10-2-76-44.ec2.internal.out
[2022-04-19:05:51:

In [7]:
%%time
xgb_predictor = xgb.deploy(initial_instance_count=3, instance_type='ml.m5.xlarge')

------!CPU times: user 105 ms, sys: 9.11 ms, total: 114 ms
Wall time: 3min 1s


In [8]:
from IPython import display
from sagemaker.serializers import CSVSerializer
xgb_predictor.serializer = CSVSerializer()

def get_prediction_row (features):
    values = xgb_predictor.predict(features).decode('utf-8')
    output = [f'{float(value):0.3g}' for value in values.split(',')]
    return pd.DataFrame(output, columns = ['Pred'])


def get_prediction_per_all(df):
    step = 10000
    prediction_set = pd.DataFrame()
    for i in range(0,len(df),step):
        feature_cols = [col for col in df.columns if 'Feature_' in col]
        feature_sets = df.loc[i:i + (step *1)-1, feature_cols]
        input_values = feature_sets.values.tolist()
        output_predictions = get_prediction_row(input_values)
        prediction_set = pd.concat([prediction_set, output_predictions],ignore_index=True)
        display.clear_output(wait=True)
        display.display(i)
    return prediction_set

In [9]:
df_test = wr.s3.read_csv(f's3://ads508team7/AfterFeatureEngr/test/data.csv')
df_validation = wr.s3.read_csv(f's3://ads508team7/AfterFeatureEngr/validate/data.csv')

prediction_set_test = get_prediction_per_all(df_test)  
df_pred_test = df_test.join(prediction_set_test)
wr.s3.to_csv(df_pred_test, 's3://ads508team7/jump_start/pred_test/data.csv', index = False)

prediction_set_valid = get_prediction_per_all(df_validation)   
df_pred_valid = df_test.join(prediction_set_valid)
wr.s3.to_csv(df_pred_valid, 's3://ads508team7/jump_start/pred_valid/data.csv', index = False)

3130000

{'paths': ['s3://ads508team7/jump_start/pred_valid/data.csv'],
 'partitions_values': {}}